<a href="https://colab.research.google.com/github/vicentcamison/idal_ia3/blob/main/5%20Procesado%20del%20lenguaje%20natural/Sesion%202/NLP_06e_Extraccio%CC%81n_de_caracteri%CC%81sticas_avanzadas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracción de características avanzadas
Aquí vamos a utilizar los datos generados mediante el pre-procesado de los textos medante librerías de NLP para sacar características avanzadas (a nivel de documento).

## Detección de entidades
El Reconocimiento de entidades nombradas (NER, *Named Entity Recognition*) es una tarea de extracción de información que busca localizar y clasificar en categorías predefinidas, como personas, organizaciones, lugares, expresiones de tiempo y cantidades, las entidades nombradas encontradas en un texto.\
La librería `spaCy` identifica ciertas entidades propias que aparecen en el texto. Podemos acceder a las entidades de un documento a través de su atributo `doc.ents`.

In [ ]:
import spacy
from spacy import displacy

nlp = spacy.load("es_core_news_sm")

In [ ]:
doc = nlp("El gran escritor Miguel de Cervantes nació en Alcalá de Henares")

In [ ]:
entidades = [e for e in doc.ents]
entidades

Podemos ver visualmente las entidades detectadas:

In [ ]:
displacy.render(doc, style='ent', jupyter=True)

Internamente, cada entidad se trata en `spaCy` como un *span* del texto. Este *span* viene definido por una marca en los *tokens* del objeto `Doc` siguiendo el esquema BIO (Begin, In, Out) y su etiqueta.

In [ ]:
type(entidades[0])

In [ ]:
datos = map(lambda t: {'token': t.orth_,
                       'POS': t.pos_,
                       'ent_iob': t.ent_iob_,
                       'ent_type': t.ent_type_
                      }, doc)

pd.DataFrame(datos)

Podemos contar las entidades del texto como una característica descriptiva del mismo:

In [ ]:
text_raw = open('cañas y barro.txt', encoding="utf8").read()
libro = nlp(text_raw)

In [ ]:
len(libro)

In [ ]:
len(libro.ents)

Estos son los distintos tipos de entidades que aparecen en el libro:  

In [ ]:
{w.label_:spacy.explain(w.label_) for w in libro.ents}

In [ ]:
from collections import Counter
Counter([e.label_ for e in libro.ents]).most_common()

Se puede hacer un filtrado/limpieza de las entidades detectadas con la librería `textaCy` (https://textacy.readthedocs.io/en/latest/api_reference/extract.html#textacy.extract.basics.entities)

In [ ]:
from textacy import extract

entidades = extract.basics.entities(libro, exclude_types="MISC")

In [ ]:
type(entidades)

### Ejercicio 1
Cuenta las entidades de cada tipo que se han detectado después del filtrado:

### Ejercicio 2
Lista las 10 entidades de tipo `PER` más frecuentes que aparecen en la novela:

Se puede utilizar el análisis de dependencias, el análisis morfológico (POS) y la detección de entidades para extraer ciertos términos del texto.\
Por ejemplo, vamos a extraer los adjetivos más utilizados por Blasco Ibáñez para describir cada personaje. Para eso, hacemos un barrido de todas las frases en las que aparece el personaje y buscamos los tokens de tipo `adj` que dependen gramaticalmente de la entidad propia del personaje.

In [ ]:
def adjectivesDescribingCharacters(text, character):
    sents = [sent for sent in text.sents if character in sent.text]
    adjectives = []
    for sent in sents: 
        for word in sent: 
            if character in word.text:
                for child in word.children: 
                    if child.pos_ == 'ADJ': 
                        adjectives.append(child.text.strip())
    return Counter(adjectives).most_common(10)

In [ ]:
adjectivesDescribingCharacters(libro, "Tonet")

## Extracción de palabras clave
Se puede obtener un listado de las palabras más importantes de un documento. Usos:
- Creación de índices para una búsqueda de documentos
- Clasificación (*clustering*) no supervisado de documentos
- Recomendadores
- Generación automática de resúmenes de texto\

La librería `textaCy` ofrece distintos algoritmos de extracción de palabras clave: https://textacy.readthedocs.io/en/latest/api_reference/extract.html#module-textacy.extract.keyterms

In [ ]:
with open('articulo.txt', 'r') as f:
    texto = f.read()

In [ ]:
texto

In [ ]:
doc = nlp(texto)
len(doc)

In [ ]:
from textacy.extract import keyterms as kt

kt.textrank(doc, normalize="lemma", topn=10)

In [ ]:
kt.sgrank(doc, normalize="lower", topn=0.1)